# Random forest classification

## Single-node scikit-learn

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/1200px-Scikit_learn_logo_small.svg.png" width="300">

In [ ]:
import os

MODEL_PATH = 'models'
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)
    
numeric_feat = [
    'pickup_weekday', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'PULocationID', 
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'high_tip'

# Load data and feature engineering

Load a full month for this exercise

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")

taxi = pd.read_csv(
    'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv',
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
)

In [ ]:
print(f'Num rows: {len(taxi)}, Size: {taxi.memory_usage(deep=True).sum() / 1e6} MB')

In [ ]:
def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
    df['high_tip'] = (df['tip_fraction'] > 0.2) # class label
    
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [y_col]].astype(float).fillna(-1)
    
    return df
    
taxi_train = prep_df(taxi)

In [ ]:
taxi_train.high_tip.value_counts()

In [ ]:
taxi_train.head()

# Train model

Setting `n_jobs=-1` tells scikit-learn to use all available cores on this machine to train models. Note that scikit-learn does NOT use the GPU, its using CPU cores here.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)

In [ ]:
%%time
_ = rfc.fit(taxi_train[features], taxi_train[y_col])

## Save model

In [ ]:
import cloudpickle

with open(f'{MODEL_PATH}/random_forest_scikit.pkl', 'wb') as f:
    cloudpickle.dump(rfc, f)

## Calculate metrics on test set

Use a different month for test set

In [ ]:
taxi_test = pd.read_csv(
    'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-02.csv',
    parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime']
)

taxi_test = prep_df(taxi_test)

In [ ]:
from sklearn.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[:, 1]
roc_auc_score(taxi_test[y_col], preds)